In [1]:
import numpy as np
import networkx as nx
from dotmotif import Motif, GrandIsoExecutor
import pandas
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
from scipy.stats import ranksums
import statsmodels.stats.multitest as smm


c:\Users\saika\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\saika\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
c:\Users\saika\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [40]:
import sys 
#sys.path.append("C://Users//saika//Downloads")
sys.path.append("C://Users//saika//OneDrive//Documents//LSMMData-main (2)//LSMMData-main//src//lsmm_data")

In [2]:
pip install update lsmm_data-0.0.6-py2.py3-none-any.whl

Processing c:\users\saika\downloads\assembly_paper\lsmm_data-0.0.6-py2.py3-none-any.whl
lsmm-data is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from lsmm_data import LSMMData
import json

with open('v1dd_data_structures_information.json') as f:
    loaded_json = json.load(f)
my_data = LSMMData.LSMMData(loaded_json)
tables = my_data.data
params = my_data.params
dirs = my_data.dirs
mappings = my_data.mappings
 
print(tables['structural']['pre_cell'])
print(tables['structural']['post_cell'])
print(tables['structural']['synapse'])
 
pt_root_id = tables['structural']['pre_cell'].iloc[1]['pt_root_id']
#print("Connectome Index.  Should be 41:", mappings['pt_root_id_to_connectome_index'][pt_root_id])


Filtering to 10 cells per type
Generating Connectome...


100%|██████████| 132646/132646 [01:52<00:00, 1183.79it/s]


Generating Connectome...


100%|██████████| 132646/132646 [01:55<00:00, 1144.04it/s]


     index      id          pt_root_id  pt_position_x  pt_position_y  \
0        9  293764  864691132625300120         878665         697702   
1       41  295576  864691132710501931         842193         737122   
2       62  278405  864691132830575876         737898         601710   
3       85  307952  864691132999200949         867646         840563   
4       93  366542  864691132834421640         923130         760946   
..     ...     ...                 ...            ...            ...   
812   8764  371302  864691132849454803         958050         629957   
813   8769  385115  864691132679146333         975354         850418   
814   8773  383491  864691132763271347         928872         797418   
815   8790  351167  864691132579592205         927863         548865   
816   8794  277788  864691132653866455         853833         547235   

     pt_position_z classification_system cell_type status_axon  \
0           245475            excitatory        4P    extended   
1  

In [9]:
tables['structural']['summed_size_connectome']

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0., 644.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 306.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [4]:
###NOT NEEDED IN LATEST VERSION
'''def calc_ground_truth(pre_cells, synapse, post_cells, use_sizes=False):
    pre_cell_pt_index = pre_cells['pt_root_id']
    post_cell_pt_index = post_cells['pt_root_id']
    synapses = synapse[(synapse['pre_pt_root_id'].isin(pre_cells['pt_root_id'])) & (synapse['post_pt_root_id'].isin(post_cells['pt_root_id']))]
    pre_cell_pt_index = pre_cell_pt_index.to_list()
    post_cell_pt_index = post_cell_pt_index.to_list()

    synapse_connectome = np.zeros((len(pre_cells), len(post_cells)))
    for synapse in tqdm(range(len(synapses))):
        preidx = pre_cell_pt_index.index(synapses.iloc[synapse]['pre_pt_root_id'])
        postidx = post_cell_pt_index.index(synapses.iloc[synapse]['post_pt_root_id'])
        if use_sizes:
            synapse_connectome[preidx,postidx] += synapses.iloc[synapse]['size']
        else:
            synapse_connectome[preidx,postidx] += 1
    return synapse_connectome'''

In [10]:

cell_table = tables['structural']['pre_cell']

weight_matrix = tables['structural']['summed_size_connectome']

# # # cell_table = cell_table.query('soma_layer == "23"')[['pt_root_id', 'soma_layer']].reset_index()
# #cell_table['connectome_index'] = cell_table['conn']

# # cell_table['connectome_index'] = cell_table.index
# # cell_table = cell_table.query('cell_type == "PYC"')[['connectome_index', 'pt_root_id', 'soma_layer']].reset_index()

# synapse_table = tables['structural']['synapse'] #pandas.read_feather('C://Users//saika//Downloads//v1dd_with_additional_coregistered_neurons_Julian_latest_now//v1dd_with_additional_coregistered_neurons_Julian_latest//additional_coregistered_neurons//synapse_table_v1dd_shortened_additional_cells.feather')
# weight_matrix = calc_ground_truth(cell_table, synapse_table, cell_table, use_sizes=True)
# adjacency_matrix = weight_matrix.clip(0,1)
# np.fill_diagonal(weight_matrix, 0)
# #pyr_graph = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)

In [11]:
weight_matrix

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0., 644.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 306.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [12]:
mappings.keys()

dict_keys(['functional_indexes_by_assembly', 'assemblies_by_functional_index', 'pt_root_id_to_connectome_index', 'connectome_index_to_root_id', 'roi_id_to_pt_root_id', 'functional_index_to_roi_id', 'assemblies_by_roi_id', 'assemblies_by_pt_root_id', 'assemblies_by_connectome_index', 'connectome_indexes_by_assembly', 'pt_root_ids_by_assembly', 'roi_ids_by_assembly'])

In [13]:
cell_table

,index,id,pt_root_id,pt_position_x,pt_position_y,pt_position_z,classification_system,cell_type,status_axon,pt_position_x_trafo,...,dist_pia,dist_1-23,dist_23-4,dist_4-5,dist_5-6,dist_WM,full_dendrite,cell_type_v2,connectome_index,pt_position
0,9,293764,864691132625300120,878665,697702,245475,excitatory,4P,extended,-17654.153169,...,348299.261582,248299.261582,78299.261582,51700.738418,201700.738418,401700.738418,True,4P,0,"[878665, 697702, 245475]"
1,41,295576,864691132710501931,842193,737122,325260,inhibitory,ProxTC,extended,-67856.489452,...,407818.941194,307818.941194,137818.941194,7818.941194,142181.058806,342181.058806,True,ProxTC-L5,1,"[842193, 737122, 325260]"
2,62,278405,864691132830575876,737898,601710,294075,inhibitory,DistTC,extended,-145995.980100,...,269521.726886,169521.726886,478.273114,130478.273114,280478.273114,480478.273114,True,DistTC-L23,2,"[737898, 601710, 294075]"
3,85,307952,864691132999200949,867646,840563,262935,excitatory,5P,extended,-69027.465946,...,490673.563262,390673.563262,220673.563262,90673.563262,59326.436738,259326.436738,True,5P,3,"[867646, 840563, 262935]"
4,93,366542,864691132834421640,923130,760946,173250,inhibitory,DistTC,extended,17492.220187,...,389679.613276,289679.613276,119679.613276,10320.386724,160320.386724,360320.386724,False,DistTC-L4,4,"[923130, 760946, 173250]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,8764,371302,864691132849454803,958050,629957,382545,inhibitory,ProxTC,extended,67043.559779,...,319323.746587,219323.746587,49323.746587,80676.253413,230676.253413,430676.253413,True,ProxTC-L4,812,"[958050, 629957, 382545]"
813,8769,385115,864691132679146333,975354,850418,394650,excitatory,6P,extended,17544.597851,...,534890.730711,434890.730711,264890.730711,134890.730711,15109.269289,215109.269289,True,6P,813,"[975354, 850418, 394650]"
814,8773,383491,864691132763271347,928872,797418,316125,excitatory,5P,extended,3578.299784,...,463037.004450,363037.004450,193037.004450,63037.004450,86962.995550,286962.995550,True,5P,814,"[928872, 797418, 316125]"
815,8790,351167,864691132579592205,927863,548865,192105,inhibitory,ProxTC,extended,51726.443534,...,190364.527692,90364.527692,79635.472308,209635.472308,359635.472308,559635.472308,False,ProxTC-L23,815,"[927863, 548865, 192105]"


In [14]:
l1 = cell_table.query('layer  == "L1"')
l23 = cell_table.query('layer  == "L23"')
l4 = cell_table.query('layer  == "L4"')
l5 = cell_table.query('layer  == "L5"')
l6 = cell_table.query('layer  == "L6"')

In [34]:
min(l1['pt_position_x_trafo'])

-137821.95442776743

In [16]:
for idx, sublist in enumerate(l1['pt_position_y_trafo']):
     pos2 = sublist  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
     print(pos2)

99838.77883754131
58961.99880086941
99175.15719302866
43268.32253312591
83819.7271757146
5830.406338428686
85954.93878029182
100076.99956833453
61105.14227848984
71061.06565542793
34394.087257864776
65192.63637324484
39898.780937435244
28519.937599981113
25903.39012702956
100855.89480013207
65592.19688795751
96612.73911140962
55894.26386791371
65638.75609179967
45349.79481681152
68549.42047879384
7719.367638640394
59390.9013318725
69995.44465419816
30886.232349149283
79799.64946418333
87750.19411825408
93379.3723602891
95874.69079827287
75481.37106285749
95104.47333817104
37917.882306254374
23087.659459281156
38328.84109988064
66177.22047347222
90589.10397215401
89018.20228430603


In [42]:
max_p2_l1 = float('-inf')
for idx, sublist in enumerate(l1['pt_position_y_trafo']):
    pos2 = sublist/1000  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    if pos2 > max_p2_l1:
        max_p2_l1 = pos2
print("L1:", max_p2_l1)

max_p2_l23 = float('-inf')
for idx, sublist in enumerate(l23['pt_position_y_trafo']):
    pos2 = sublist/1000  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    if pos2 > max_p2_l23:
        max_p2_l23 = pos2
print("L23:",max_p2_l23)

max_p2_l4 = float('-inf')
for idx, sublist in enumerate(l4['pt_position_y_trafo']):
    pos2 = sublist/1000 # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    if pos2 > max_p2_l4:
        max_p2_l4 = pos2
print("L4:",max_p2_l4)

max_p2_l5 = float('-inf')
for idx, sublist in enumerate(l5['pt_position_y_trafo']):
    pos2 = sublist/1000  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    if pos2 > max_p2_l5:
        max_p2_l5 = pos2
print("L5:",max_p2_l5)

max_p2_l6 = float('-inf')
for idx, sublist in enumerate(l6['pt_position_y_trafo']):
    pos2 = sublist/1000  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    if pos2 > max_p2_l6:
        max_p2_l6 = pos2
print("L6:",max_p2_l6)

L1: 100.85589480013206
L23: 274.9746804750402
L4: 407.5061989400466
L5: 570.16040905315
L6: 780.8385364081698


In [43]:
max_p2_l1

100.85589480013206

In [317]:
'''coords = np.load("C://Users//saika//Downloads//transformed_coordinates.npy")'''

In [ ]:
'''for idx,connections  in enumerate(coords):
    print(connections)'''

In [13]:
weight_matrix

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0., 644.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 306.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [18]:
#checking psd sizes
summed_psd_size = []
for idx,connections  in enumerate(weight_matrix):
    for conn_idx,conn_value in enumerate(connections):
        summed_psd_size.append(conn_value)

In [19]:
np.max(summed_psd_size)

53280.0

cell_table


In [20]:
np.max(weight_matrix)

53280.0

In [ ]:
'''for idx,connections  in enumerate(weight_matrix):
    for conn_idx,conn_value in enumerate(connections):
        print(conn_idx, conn_value)'''

In [44]:
import matplotlib as mpl

def colorFader(c1, c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [45]:
for idx, pos1, pos2 in zip(cell_table['connectome_index'], cell_table['pt_position_x_trafo'], cell_table['pt_position_y_trafo']):
   # pos1, pos2, pos3 = sublist  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    # pos1_rounded = round(pos1, 2)
    # pos2_rounded = round(pos2, 2)
    pos = [pos1/1000, pos2/1000]
    print(pos)

[-17.654153168534894, 354.663149819886]
[-67.85648945208777, 415.8291471175572]
[-145.99598010020043, 274.7942345083574]
[-69.02746594583235, 503.38019287626696]
[17.49222018713105, 397.1541133097775]
[-18.184720639622242, 190.41408362748876]
[-42.82235135709832, 142.72503055448465]
[-7.791778786625675, 330.5880075939716]
[-21.10589889608785, 388.20688939055185]
[-103.29558946108513, 232.84670337755875]
[19.932217918203815, 498.68529203658926]
[-109.67057604224307, 545.5381513494699]
[31.801251247086547, 417.6178426637854]
[78.04969379082794, 332.6574785652873]
[44.92297027255013, 99.83877883754131]
[12.834739112501701, 267.78812472947646]
[10.134704494913425, 314.71733806147387]
[0.5273306710736337, 58.96199880086941]
[-65.97683058618725, 345.7707989867011]
[28.64256258571777, 112.43626864446837]
[-15.192691011364676, 390.61024254153483]
[33.25118184662824, 679.458840006381]
[-33.899412591799205, 158.0393803017652]
[14.260540607301698, 213.69582865675764]
[-29.26443397266496, 566.6211

In [46]:

c1 = "xkcd:yellowgreen"
c2 = "cyan"


G = nx.Graph()

for idx, pos1, pos2 in zip(cell_table['connectome_index'], cell_table['pt_position_x_trafo'], cell_table['pt_position_y_trafo']):
   # pos1, pos2, pos3 = sublist  # Unpack the elements in the sublist #use co-ordinates from tables and check if pt_root_id matches
    # pos1_rounded = round(pos1, 2)
    # pos2_rounded = round(pos2, 2)
    G.add_node(idx, pos = [pos1/1000, pos2/1000])  


for idx,connections  in enumerate(weight_matrix):
    for conn_idx,conn_value in enumerate(connections):
        if 15000 < conn_value <= 18000:
            G.add_edge(idx,conn_idx, weight = 0.2, color = colorFader(c1, c2, 0), label='Connection 1')
        elif 18000 < conn_value <= 30000:
            G.add_edge(idx,conn_idx, weight = 0.8, color = colorFader(c1, c2, 1/4), label='Connection 2')
        elif 30000 < conn_value <= 45000:
            G.add_edge(idx,conn_idx, weight = 1.2, color = colorFader(c1, c2, 2/4), label='Connection 3')
        elif conn_value> 45000:
            G.add_edge(idx,conn_idx, weight = 2, color = colorFader(c1, c2, 3/4), label='Connection 4')
   # if idx>0:
   #     G.add_edge(idx-1, idx)  # Adding edge between consecutive nodes




In [47]:
colors = nx.get_edge_attributes(G,'color').values()
weights = nx.get_edge_attributes(G,'weight').values()


In [48]:
color_map = []
assembly_map = []
idx = []
no_assembly = ['No A']
for node in G.nodes():
    if node in  mappings['assemblies_by_connectome_index'].keys():
                   val = mappings['assemblies_by_connectome_index'][node]
                   if val == no_assembly:
                         
                          color_map.append('red')
                          #assembly_map.append('black')
                   else:
                            #print(nik)
                            color_map.append('blue')  
                            
            
            #idx.append(node)
    else:
            color_map.append('grey')
            #idx.append(node)


In [49]:
len(color_map)

817

In [50]:
pos = nx.get_node_attributes(G, 'pos')


In [52]:
from matplotlib.lines import Line2D
from matplotlib.patches import FancyBboxPatch
import matplotlib as mpl


In [ ]:
##TRIAL FIGURE
'''
fig, ax = plt.subplots()



nx.draw(G, pos,
        width=list(weights),
        edge_color=colors, 
        with_labels=False,node_size=4,
        node_color=color_map, ax= ax)
limits=plt.axis('on') # turns on axis



# Create custom legend handles for nodes
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label='Assembly cells'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='Non-assembly cells'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgrey', markersize=10, label='Unregistered cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='green', markersize=10, label='Assembly cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='b', markersize=10, label='Non-assembly cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='y', markersize=10, label='Assembly cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='m', markersize=10, label='Non-assembly cells')
]

# Add the legend to the plot
node_legend = ax.legend(handles=legend_elements, loc='upper left')

# Create custom legend handles for edges
edge_legend_elements = [
    Line2D([0], [0], color='lightgreen', lw=0.5),
    Line2D([0], [0], color='b', lw=1),
    Line2D([0], [0], color='y', lw=1.5),
    Line2D([0], [0], color='m', lw=2)
]

# Add the legend to the plot
edge_legend = ax.legend(handles=edge_legend_elements, loc='lower right', title='Synapse strength: low to high',title_fontsize = 6.5, bbox_to_anchor=(1, 0))

# Show the plot
#ax.invert_yaxis()
ax.add_artist(node_legend)  # Add the node legend first
ax.add_artist(edge_legend)

ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)

plt.xlabel('pt position X')
plt.ylabel('pt position Y')
plt.title('Connectome Network')
plt.show()'''

In [55]:
import seaborn as sns

In [1]:


fig, ax = plt.subplots(figsize=(12,8))


nx.draw(G, pos,
        width=list(weights),
        edge_color=colors, 
        with_labels=False,node_size=10,
        node_color=color_map, ax= ax)
limits=plt.axis('on') # turns on axis



#-------Add layers to plot----------
y_values = [max_p2_l1, max_p2_l23,max_p2_l4,  max_p2_l5,  max_p2_l6]
#y_values= [100, 270, 400,  550, 750]
labels = ['Layer 1', 'Layer 23', 'Layer 4', 'Layer 5', 'Layer 6']

ax = plt.gca()
offset = 1.5
# Add horizontal lines and labels
for y, label in zip(y_values, labels):
    ax.axhline(y=y, color='chocolate', linestyle='--', linewidth=1)
    #x = ax.get_x()
    x_min, x_max = ax.get_xlim()
    ax.text(x_max - offset,y - offset, label, color='black', fontsize=12, ha='right', va='bottom')

#---------------------------------

# Create custom legend handles for nodes
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=8, label='Assembly cells'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8, label='Non-assembly cells'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='grey', markersize=8, label='Not co-registered cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='green', markersize=10, label='Assembly cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='b', markersize=10, label='Non-assembly cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='y', markersize=10, label='Assembly cells'),
    # Line2D([0], [0], marker='_', color='w', markerfacecolor='m', markersize=10, label='Non-assembly cells')
]

# Add the legend to the plot
node_legend = ax.legend(handles=legend_elements, loc='upper left', fontsize='12', fancybox = False)  #fancybox=false to prevent rounded corners
#node_legend.set_ed

# Create custom legend handles for edges
edge_legend_elements = [
    Line2D([0], [0], color=colorFader(c1, c2, 3/4), lw=2),
    Line2D([0], [0], color=colorFader(c1, c2, 2/4),lw=1.5),
    Line2D([0], [0], color=colorFader(c1, c2, 1/4),lw=1),
     Line2D([0], [0], color=colorFader(c1, c2, 0), lw=0.5)
]




#edge_legend = ax.legend(handles=edge_legend_elements, loc='lower left',title='Synapse Strength (PSD)',fontsize = 6.5,title_fontsize = 6.5)

    # Line2D([0], [0], color='yellow', label = '15000 < synapse strength <= 18000' ,  lw=0.5),
    # Line2D([0], [0], color='orange',label = '18000 < synapse strength <= 30000' ,lw=1),
    # Line2D([0], [0], color='cyan',label = '30000  < synapse strength <= 45000' ,lw=1.5),
    # Line2D([0], [0], color='m', label = '45000 < synapse strength', lw=2)


#Add the labels for edge legend
edge_labels = ['45000<  <   55000', '30000<  <= 45000', '18000<  <= 30000', '15000<   <= 18000' ]

### Create the edge legend - PSD Volume BOX ####

# Adjust the position for the bottom left corner (using figure coordinates)
x_coord = 0.18  # Value to move to the left(move horizontal)
y_coord = 0.125  # Value to adjust the distance from bottom
y_step_size = 0.05  # To maintain the vertical space between the four lines

title = (r"PSD Volume in nm$^{3}$")
fig.text(x_coord, y_coord + 0.18, title, fontsize=10.5, verticalalignment='center', horizontalalignment='left', fontweight='normal')

for i, (legend_element, label) in enumerate(zip(edge_legend_elements, edge_labels)):
    y_pos = y_coord + i * y_step_size
    
    #Using fig. instead of ax. allows it to align with the figure. Prevents the shrinking of the main figure.

    # Position text before the legend
    fig.text(x_coord - 0.012, y_pos, label.split(' ')[0], fontsize=9.5, verticalalignment='center', horizontalalignment='right')
    
    # Position the legend element
    fig.add_artist(Line2D([x_coord, x_coord + 0.1], [y_pos, y_pos], color=legend_element.get_color(), lw=legend_element.get_linewidth()))
    
    # Position text after the legend
    fig.text(x_coord + 0.11, y_pos, ' '.join(label.split(' ')[2:]), fontsize=9.5, verticalalignment='center', horizontalalignment='left')

#create the boundary box for edge legends
box_x = x_coord + 0.048 # add to move to right and subtract to move left
box_y = y_coord + 0.089 # add to move above the bottom
box_width = 0.0126 #adjust width of the box
box_height = 0.014 #adjust the height of the box


box = FancyBboxPatch((box_x, box_y), box_width, box_height,
                     boxstyle="square,pad=0.1", edgecolor='lightgrey', facecolor='white', linewidth=1.5,
                     transform=fig.transFigure, zorder=1)
fig.patches.append(box)



#########trial method for text legend combo - did not work (use label.split() instead)#############
# ax.annotate('1: Conn 1: ', xy=(1.05, 0.9), xycoords='axes fraction', color='black', fontsize=12, ha='left', va='center'),
# ax.annotate('-'*2, xy=(1.24, 0.9), xycoords='axes fraction', color='red', fontsize=12, ha='left', va='center'),
# ax.annotate('1: Conn 1: ', xy=(1.5, 0.9), xycoords='axes fraction', color='black', fontsize=12, ha='left', va='center'),


# Show the plot

#add both node and edge legends
ax.add_artist(node_legend)  # Add the node legend first
#ax.add_artist(edge_legend)

#invert axis to get Layer 6 at bottom
ax.invert_yaxis()
#ax.invert_xaxis()

ax.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
ax.legend(markerfirst = False)
# Customize with Seaborn styles if needed
sns.set_theme(style="white") 

plt.xlabel(r'pt position X ($\mu$m)') 
plt.ylabel(r'pt position Y ($\mu$m)')
plt.title('Connectome Network')
plt.autoscale()
plt.savefig('connectivity_plot_corrected_sns.png', dpi=1200) 
plt.show()

NameError: name 'plt' is not defined